In [1]:
%%capture
!pip install tensorboardX simpletransformers sklearn

In [2]:
!cat /usr/local/cuda/version.txt
!pip install pydash
!pip install torch==1.6.0
#conda install pytorch==1.1.0 torchvision==0.3.0 cudatoolkit=10.0 -c pytorch
#https://pytorch.org/get-started/previous-versions/

CUDA Version 10.0.130
     |████████████████████████████████| 748.8 MB 6.0 kB/s              


In [3]:
!rm -rf outputs

In [17]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
import logging
import sklearn
from tqdm import tqdm
import pandas as pd

df = pd.read_csv("downloads/47k_train_no_acl.csv")
len(df)

41561

In [18]:
blacklist=["abstract", "discussion", "background", "summary", "rationale", "introduction", "purpose", "objective" ]
blacklist += [t + ":" for t in blacklist]
def remove_special_words(text, blacklist=blacklist ):
    words = text.split(" ")
    try:
        first=words[0]
    except:
        return text
    
    if any(first.strip().lower() in string for string in blacklist):
        return " ".join(words[1:]).strip()
    return text

remove_special_words("ABSTRACTFigure   The northern corn rootworm"), remove_special_words("ABSTRACT   The northern corn rootworm")

('ABSTRACTFigure   The northern corn rootworm', 'The northern corn rootworm')

In [19]:
df["text"] = df["text"].map(remove_special_words)
df["source"].unique()

array(['predicts_acl_pm2500_vocabed', 'Oct1_clinical_studies_pm',
       'oct3_labels', 'labels_oct7',
       ' limitation|problem ... with ... l=be that', 'a ... drawback of',
       'a {negative_adjectives} ...  {problem_nouns}', 'pm_OBJECTIVE',
       'pm_METHODS', 'pm_RESULTS', 'pm_CONCLUSIONS', 'biology1k_050522',
       'psych1k_100522'], dtype=object)

In [20]:
#df[df.source == "predicts_acl_pm2500_vocabed"]["text"]

In [21]:
df.source.unique()

array(['predicts_acl_pm2500_vocabed', 'Oct1_clinical_studies_pm',
       'oct3_labels', 'labels_oct7',
       ' limitation|problem ... with ... l=be that', 'a ... drawback of',
       'a {negative_adjectives} ...  {problem_nouns}', 'pm_OBJECTIVE',
       'pm_METHODS', 'pm_RESULTS', 'pm_CONCLUSIONS', 'biology1k_050522',
       'psych1k_100522'], dtype=object)

In [22]:
#downsample medical data in training set
df = df[df["source"].isin(["oct3_labels", "labels_oct7", "psych1k_100522", "biology1k_050522", "predicts_acl_pm2500_vocabed", 'pm_CONCLUSIONS', 'biology1k_050522'])]

In [23]:
df= df[["text", "labels"]]

In [24]:
df =df.dropna()
df = df.rename(columns={"labels": "label"})

In [25]:
df= df[["text", "label"]]
df.label = df.label.map(int)

In [26]:
df

,text,label
0,"While often idiopathic, anterior ischemic opti...",0
1,Primary cutaneous B-cell lymphomas (PCBCLs) ar...,0
2,The insufflation of gas during colonoscopy lea...,0
3,The aim of the present study is to investigate...,0
4,The aim of this study was to assess predictive...,0
...,...,...
41556,Mobbing is actually a social problem.,0
41557,Students starting off on a tertiary education ...,1
41558,Many individuals with vision impairment experi...,1
41559,Considering China is facing a precipitous decl...,0


In [27]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

df2 = df.sample(frac=1)
# Train and Evaluation data needs to be in a Pandas Dataframe of two columns. 
#The first column is the text with type str, and the second column is the label with type int.
size = len(df2)
train_cutoff = round(size*0.8)
train_df= df2[:train_cutoff]
eval_df = df2[train_cutoff:]

model_args = ClassificationArgs(save_steps=-1, save_model_every_epoch=False, sliding_window=True)

model = ClassificationModel('distilbert', 'distilbert-base-uncased',args=model_args, use_cuda=True) # You can set class weights by using the optional weight argument

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'classifi

In [28]:
df[df.label == 1]

,text,label
5,Although spatial ability and mathematics perfo...,1
12,Postoperative liver failure is a serious compl...,1
17,Although previous correlational studies have s...,1
18,As exchange counting poses difficulty for chil...,1
20,Chronic intestinal pseudo-obstruction (CIPO) i...,1
...,...,...
41554,Excessive consumption of unhealthy foods is a ...,1
41555,Objectives: A major challenge in cognitive agi...,1
41557,Students starting off on a tertiary education ...,1
41558,Many individuals with vision impairment experi...,1


In [29]:
# Train the model
model.train_model(train_df, overwrite_output_dir=True)
# Evaluate the model, u
result, model_outputs, wrong_predictions = model.eval_model(eval_df, acc=sklearn.metrics.accuracy_score)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/simpletransformers/classification/classification_model.py:602: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_model: Sliding window enabled


  0%|          | 0/6134 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: 6147 features created from 6134 samples.
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/769 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: Training of distilbert model complete. Saved to outputs/.
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/simpletransformers/classification/classification_model.py:1443: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_model: Sliding window enabled


  0%|          | 0/1533 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: 1533 features created from 1533 samples.


Running Evaluation:   0%|          | 0/192 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model:{'mcc': 0.7774264713536183, 'tp': 234, 'tn': 1195, 'fp': 63, 'fn': 41, 'acc': 0.9321591650358774, 'eval_loss': 0.20802968784604067}


In [30]:
result

{'mcc': 0.7774264713536183,
 'tp': 234,
 'tn': 1195,
 'fp': 63,
 'fn': 41,
 'acc': 0.9321591650358774,
 'eval_loss': 0.20802968784604067}

In [31]:
import pandas as pd

In [ ]:
dfx = pd.read_csv("downloads/s2c_5M_bioadjacent.csv",  lineterminator='\n', error_bad_lines=False)

In [ ]:
dfx = dfx.dropna(subset=["text"])
dfx=dfx[dfx["text"].str.len() > 15]

dfx.head()

#dfx = dfx[dfx.fields.isin(["Medicine", 'Environmental Science', 'Sociology', 'Psychology', 'Geology', 'Biology', 'Materials Science', 'Chemistry'])]
dfx = dfx[dfx.fields.isin(["Medicine"])]
dfx["text"] = dfx["text"].map(remove_special_words)

len(dfx)

1178184

In [38]:
dfx["text"] = dfx["text"].astype(str)
dfx = dfx.dropna(subset=["doi"])


In [35]:
#dfx = dfx.sample(frac=0.2)

In [39]:
len(dfx)

900121

In [40]:
#dfx.to_csv("downloads/sample_bioadjacent600k.csv", index = False, header=True)

In [47]:
predicts_path = 'downloads/predicts_backup_5M.txt'

In [41]:
def stepwise(step, df, f):
    l = []
    for idx in range(step, len(df)+step-1, step):
        print(idx-step, idx)# dataframe[idx-step:idx])
        sl = df[idx-step:idx]
        batch = model.predict(list(sl))
        l = l + list(batch[0])
        with open(predicts_path, 'w') as f:
            for item in l:
                f.write("%s\n" % item)
    return l

In [ ]:
predicts = stepwise(40000, dfx["text"], model.predict)

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_model: Sliding window enabled


0 40000


  0%|          | 0/40000 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: 40000 features created from 40000 samples.


  0%|          | 0/5018 [00:00<?, ?it/s]

...

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_model: Sliding window enabled


880000 920000


  0%|          | 0/20121 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (528 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (515 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (794 > 512). Running this sequence through the model will result in indexing errors
INFO:simpletransformers.classification.classification_model: 20121 features created from 20121 samples.


  0%|          | 0/2526 [00:00<?, ?it/s]

In [45]:
dfx

,index,text,s2Url,doi,journalName,title,fieldsOfStudy,fields
0,1,Automated medication dispensing systems for ho...,https://semanticscholar.org/paper/062e9c7579ad...,10.2190/W2BT-FGXQ-QL0G-YNL9,International Journal of Health Services,Clinical or Industrial Pharmacy? Case Studies ...,['Medicine'],Medicine
1,2,"the future, the use of imaging methods to quan...",https://semanticscholar.org/paper/771723e7e2d9...,10.1016/S1052-5149(02)00023-0,Neuroimaging clinics of North America,Noncoronary and coronary atherothrombotic plaq...,['Medicine'],Medicine
3,6,BackgroundCorneal endothelium (CE) images prov...,https://semanticscholar.org/paper/984774e366d3...,10.1186/s42490-019-0003-2,BMC Biomedical Engineering,Fully convolutional architecture vs sliding-wi...,"['Medicine', 'Computer Science']",Medicine
4,7,Falsified and substandard drugs are a global h...,https://semanticscholar.org/paper/0d0467c4cccb...,10.1371/journal.pone.0090601,PLoS ONE,Technologies for Detecting Falsified and Subst...,['Medicine'],Medicine
6,11,Objectives To explore the molecular epidemiolo...,https://semanticscholar.org/paper/878a852598fb...,10.1371/journal.pone.0211261,PLoS ONE,Mutation screening in non-syndromic hearing lo...,['Medicine'],Medicine
...,...,...,...,...,...,...,...,...
5076841,4772310,9-year-old crossbred dog was presented with a ...,https://semanticscholar.org/paper/a17cd6b21405...,10.1016/j.jcpa.2013.04.005,Journal of comparative pathology,Loeffler's endocarditis and bicavity eosinophi...,['Medicine'],Medicine
5076853,4772333,To screen HCV infection in HIV infected indivi...,https://semanticscholar.org/paper/512060c86ac9...,10.5799/jmid.368504,Journal of Microbiology and Infectious Diseases,Epidemiology of Hepatitis C Virus and Human Im...,['Medicine'],Medicine
5076883,4772400,DUODENAL villous tumors are rare lesions that ...,https://semanticscholar.org/paper/cbad468d9b16...,10.1001/JAMA.1979.03290510042026,JAMA,Duodenal filling defect.,['Medicine'],Medicine
5076893,4772418,Cell mass and viability are tightly linked to ...,https://semanticscholar.org/paper/098fbc2a791a...,10.1038/s41598-021-90703-8,Scientific Reports,Analysis of methods for quantifying yeast cell...,['Medicine'],Medicine


In [48]:
with open(predicts_path, 'r') as f:
    preds = f.read().split()#f.write("%s\n" % item)
                

In [49]:
len(preds)

900121

In [ ]:
!free

In [106]:
#dfx = pd.read_csv("downloads/sample_bioadjacent600k.csv",   lineterminator='\n', error_bad_lines=False)

In [109]:
dfx.fields.value_counts(normalize=True), dd.fields.value_counts(normalize=True)

(Medicine                 0.372515
 Chemistry                0.161016
 Biology                  0.156869
 Materials Science        0.130517
 Psychology               0.078705
 Sociology                0.042472
 Geology                  0.029328
 Environmental Science    0.028579
 Name: fields, dtype: float64,
 Medicine                 0.635473
 Biology                  0.166522
 Chemistry                0.054003
 Psychology               0.053120
 Materials Science        0.033784
 Environmental Science    0.027580
 Sociology                0.016836
 Geology                  0.012683
 Name: fields, dtype: float64)

In [50]:
dfo = pd.read_csv("downloads/47k_train_no_acl.csv")


In [51]:
#dfx = dfx[:3000000]
dfx["predicts"] = preds

In [52]:
dfx["predicts"] = dfx["predicts"].map(int)

In [53]:
dfx = dfx[dfx["predicts"]==1]
dfx = dfx[dfx["text"] != "nan"]
dfx = dfx[dfx["text"].str.len() > 14]
len(dfx)

174804

In [56]:
dfx.to_csv("downloads/17June_5M_adjacent_medicine_predicts.csv", index = False, header=True)

In [57]:
dfx

,index,text,s2Url,doi,journalName,title,fieldsOfStudy,fields,predicts
4,7,Falsified and substandard drugs are a global h...,https://semanticscholar.org/paper/0d0467c4cccb...,10.1371/journal.pone.0090601,PLoS ONE,Technologies for Detecting Falsified and Subst...,['Medicine'],Medicine,1
7,13,IntroductionMcCune-Albright syndrome is a comp...,https://semanticscholar.org/paper/a28cf4e1bf12...,10.1186/1752-1947-6-32,Journal of Medical Case Reports,Pamidronic acid and cabergoline as effective l...,['Medicine'],Medicine,1
72,147,"As men grow older, testosterone (T) levels dec...",https://semanticscholar.org/paper/6616f4ba272f...,10.4103/1008-682X.122357,Asian Journal of Andrology,Sex steroids and cardiovascular disease,['Medicine'],Medicine,1
103,203,Viral infections have the potential to induce ...,https://semanticscholar.org/paper/15bdef5b0f93...,10.1007/s11901-016-0292-z,Current Hepatology Reports,HCC: Where Does HCV Therapy Play a Role?,['Medicine'],Medicine,1
123,243,Mastocytosis is a myeloid neoplasm characteriz...,https://semanticscholar.org/paper/ec9263657cf8...,10.1586/ehm.10.42,Expert Review of Hematology,"Pathogenesis, classification and treatment of ...",['Medicine'],Medicine,1
...,...,...,...,...,...,...,...,...,...
5076747,4772128,The association of a benign ovarian tumor with...,https://semanticscholar.org/paper/b25d3eec1c5c...,10.4314/WAJM.V26I3.28322,West African journal of medicine,Meigs' syndrome presenting as haemorrhagic ple...,['Medicine'],Medicine,1
5076802,4772221,The putative effects of diabetes and metabolic...,https://semanticscholar.org/paper/164e57704ccb...,10.1210/JCEM.80.4.7536202,The Journal of clinical endocrinology and meta...,Insulin-like growth factor (IGF)-I and -II and...,['Medicine'],Medicine,1
5076812,4772243,Factor V deficiency secondary to inhibitors is...,https://semanticscholar.org/paper/ca68ab337e14...,10.1155/2015/603402,Case Reports in Hematology,Refractory Epistaxis due to Severe Factor V De...,['Medicine'],Medicine,1
5076833,4772289,Recent prostate cancer screening trials have g...,https://semanticscholar.org/paper/368327b75301...,10.1101/402743,bioRxiv,A natural history model for planning prostate ...,['Medicine'],Medicine,1


In [54]:
dfx=dfx[dfx["doi"].notna()]

In [55]:
dd = dfx[~dfx["doi"].isin(list(dfo["doi"]))]
dfx=dd

In [88]:
dfx["fields"].value_counts()

Medicine                 51106
Biology                  13392
Chemistry                 4343
Psychology                4272
Materials Science         2717
Environmental Science     2218
Sociology                 1354
Geology                   1020
Name: fields, dtype: int64

In [97]:
df2 = dfx[dfx["fields"].isin(["Sociology", "Geology", "Environmental Science"])]

In [101]:
df2.to_csv("downloads/underrepresented_domains_predicts.csv", index = False, header=True)

In [102]:
df2 = dfx[dfx["fields"].isin(["Materials Science", "Psychology", "Chemistry"])]

In [105]:
df2.to_csv ("downloads/mid_represented_domains_predicts.csv", index = False, header=True)
#check that not in final110k_1605

In [ ]:
dfx.to_csv("downloads/pos_preds_12062022.csv", index = False, header=True)

In [ ]:
dfx[dfx["fieldsOfStudy"].isin(['Environmental Science', 'Sociology', 'Geology', 'Biology', 'Materials Science', 'Chemistry'])]

In [58]:
dfx

,index,text,s2Url,doi,journalName,title,fieldsOfStudy,fields,predicts
4,7,Falsified and substandard drugs are a global h...,https://semanticscholar.org/paper/0d0467c4cccb...,10.1371/journal.pone.0090601,PLoS ONE,Technologies for Detecting Falsified and Subst...,['Medicine'],Medicine,1
7,13,IntroductionMcCune-Albright syndrome is a comp...,https://semanticscholar.org/paper/a28cf4e1bf12...,10.1186/1752-1947-6-32,Journal of Medical Case Reports,Pamidronic acid and cabergoline as effective l...,['Medicine'],Medicine,1
72,147,"As men grow older, testosterone (T) levels dec...",https://semanticscholar.org/paper/6616f4ba272f...,10.4103/1008-682X.122357,Asian Journal of Andrology,Sex steroids and cardiovascular disease,['Medicine'],Medicine,1
103,203,Viral infections have the potential to induce ...,https://semanticscholar.org/paper/15bdef5b0f93...,10.1007/s11901-016-0292-z,Current Hepatology Reports,HCC: Where Does HCV Therapy Play a Role?,['Medicine'],Medicine,1
123,243,Mastocytosis is a myeloid neoplasm characteriz...,https://semanticscholar.org/paper/ec9263657cf8...,10.1586/ehm.10.42,Expert Review of Hematology,"Pathogenesis, classification and treatment of ...",['Medicine'],Medicine,1
...,...,...,...,...,...,...,...,...,...
5076747,4772128,The association of a benign ovarian tumor with...,https://semanticscholar.org/paper/b25d3eec1c5c...,10.4314/WAJM.V26I3.28322,West African journal of medicine,Meigs' syndrome presenting as haemorrhagic ple...,['Medicine'],Medicine,1
5076802,4772221,The putative effects of diabetes and metabolic...,https://semanticscholar.org/paper/164e57704ccb...,10.1210/JCEM.80.4.7536202,The Journal of clinical endocrinology and meta...,Insulin-like growth factor (IGF)-I and -II and...,['Medicine'],Medicine,1
5076812,4772243,Factor V deficiency secondary to inhibitors is...,https://semanticscholar.org/paper/ca68ab337e14...,10.1155/2015/603402,Case Reports in Hematology,Refractory Epistaxis due to Severe Factor V De...,['Medicine'],Medicine,1
5076833,4772289,Recent prostate cancer screening trials have g...,https://semanticscholar.org/paper/368327b75301...,10.1101/402743,bioRxiv,A natural history model for planning prostate ...,['Medicine'],Medicine,1


In [181]:
dt = pd.read_csv("core_pos.csv")
#d[d["first_sent"].str.contains(re.escape("|".join(d["authors"][2:-2].replace("'", "").split(","))).strip().replace('\\', ''))]
def formatStr (x):
    x = eval(x)
    x = [re.escape(v) for v in x]
    return "|".join(x)

regexs = d.authors.apply(formatStr)
matches = pd.Series()
for i, r in tqdm(d.iterrows()):
    matches.loc[i] = len(re.findall(regexs.loc[i], d.first_sent.loc[i])) > 0
dc["authors"] = dc["authors"].map(eval)
dc = dc.drop(columns=["fullText"])
dc.to_csv("core_pos_parsed.csv", index = False, header=True)
tt.to_csv("core_pos_min.csv", index = False, header=True)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:8: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
44962it [00:59, 760.29it/s] 


distilbert
'mcc': 0.7621432780395265, 'tp': 167, 'tn': 480, 'fp': 39, 'fn': 30, 'acc': 0.9036312849162011, 'eval_loss': 0.24669100075132316}

